# Heat equation tutorial

In this tutorial, we'll be solving the [heat equation](https://en.wikipedia.org/wiki/Heat_equation):

``
\frac{∂ ρcT}{∂ t} + ∇ ⋅ (-α ∇ρcT) = 0
``

where
 - `t` is time
 - `α` is the thermal diffusivity
 - `T` is the temperature
 - `ρ` is the density
 - `c` is the heat capacity
 - `ρcT` is the thermal energy

To put this in the form of ClimateMachine's `BalanceLaw`, we'll re-write the equation as:

``
\frac{∂ ρcT}{∂ t} + ∇ ⋅ (F(α, ρcT, t)) = 0
``

where
 - ``F(α, ρcT, t) = -α ∇ρcT`` is the second-order flux

with boundary conditions
 - Fixed temperature ``T_{surface}`` at ``z_{min}`` (non-zero Dirichlet)
 - No thermal flux at ``z_{min}`` (zero Neumann)

Solving these equations is broken down into the following steps:
1) Preliminary configuration
2) PDEs
3) Space discretization
4) Time discretization
5) Solver hooks / callbacks
6) Solve
7) Post-processing

# Preliminary configuration

## Loading code

First, we'll load our pre-requisites
 - load external packages:

In [1]:
using MPI
using NCDatasets
using OrderedCollections
using Plots
using StaticArrays

 - load necessary ClimateMachine modules:

In [2]:
using ClimateMachine
using ClimateMachine.Mesh.Topologies
using ClimateMachine.Mesh.Grids
using ClimateMachine.Writers
using ClimateMachine.DGmethods
using ClimateMachine.DGmethods.NumericalFluxes
using ClimateMachine.DGmethods: BalanceLaw, LocalGeometry
using ClimateMachine.MPIStateArrays
using ClimateMachine.GenericCallbacks
using ClimateMachine.ODESolvers
using ClimateMachine.VariableTemplates

 - import necessary ClimateMachine modules: (`import` indicates that we must provide implementations of these structs/methods)

In [3]:
import ClimateMachine.DGmethods:
    vars_state_auxiliary,
    vars_state_conservative,
    vars_state_gradient,
    vars_state_gradient_flux,
    source!,
    flux_second_order!,
    flux_first_order!,
    compute_gradient_argument!,
    compute_gradient_flux!,
    update_auxiliary_state!,
    nodal_update_auxiliary_state!,
    init_state_auxiliary!,
    init_state_conservative!,
    boundary_state!

## Initialization

Define the float type (`Float64` or `Float32`)

In [4]:
FT = Float64;

Initialize ClimateMachine for CPU.

In [5]:
ClimateMachine.init(; disable_gpu = true);

const clima_dir = dirname(dirname(pathof(ClimateMachine)));

Load some helper functions (soon to be incorporated into `ClimateMachine/src`)

In [6]:
include(joinpath(clima_dir, "tutorials", "Land", "helper_funcs.jl"));
include(joinpath(clima_dir, "tutorials", "Land", "plotting_funcs.jl"));

# Define the set of Partial Differential Equations (PDEs)

## Define the model

Model parameters can be stored in the particular `BalanceLaw`, in this case, a `HeatModel`:

In [7]:
Base.@kwdef struct HeatModel{FT} <: BalanceLaw
    "Heat capacity"
    ρc::FT = 1
    "Thermal diffusivity"
    α::FT = 0.01
    "Initial conditions for temperature"
    initialT::FT = 295.15
    "Bottom boundary value for temperature (Dirichlet boundary conditions)"
    T_bottom::FT = 300.0
    "Top flux (α∇ρcT) at top boundary (Neumann boundary conditions)"
    flux_top::FT = 0.0
end

Create an instance of the `HeatModel`:

In [8]:
m = HeatModel{FT}();

This model dictates the flow control, using [Dynamic Multiple Dispatch](https://en.wikipedia.org/wiki/Multiple_dispatch), for which kernels are executed.

## Define the variables

All of the methods defined below, in this section, were `import`ed in Loading code, and we must provide our own implementation as these methods are called inside `solve!` in the Solve section below. Each of these methods have a fall-back of "do nothing" if we don't, or fail to, implement these methods after `import`ing.

Specify auxiliary variables for `HeatModel`

In [9]:
vars_state_auxiliary(::HeatModel, FT) = @vars(z::FT, T::FT);

Specify state variables, the variables solved for in the PDEs, for `HeatModel`

In [10]:
vars_state_conservative(::HeatModel, FT) = @vars(ρcT::FT);

Specify state variables whose gradients are needed for `HeatModel`

In [11]:
vars_state_gradient(::HeatModel, FT) = @vars(ρcT::FT);

Specify gradient variables for `HeatModel`

In [12]:
vars_state_gradient_flux(::HeatModel, FT) = @vars(α∇ρcT::SVector{3, FT});

## Define the compute kernels

Specify the initial values in `aux::Vars`, which are available in `init_state_conservative!`. Note that
- this method is only called at `t=0`
- `aux.z` and `aux.T` are available here because we've specified `z` and `T` in `vars_state_auxiliary`

In [13]:
function init_state_auxiliary!(m::HeatModel, aux::Vars, geom::LocalGeometry)
    aux.z = geom.coord[3]
    aux.T = m.initialT
end;

Specify the initial values in `state::Vars`. Note that
- this method is only called at `t=0`
- `state.ρcT` is available here because we've specified `ρcT` in `vars_state_conservative`

In [14]:
function init_state_conservative!(
    m::HeatModel,
    state::Vars,
    aux::Vars,
    coords,
    t::Real,
)
    state.ρcT = m.ρc * aux.T
end;

The remaining methods, defined in this section, are called at every time-step in the solver by the `BalanceLaw` framework.

Overload `update_auxiliary_state!` to call `heat_eq_nodal_update_aux!`, or any other auxiliary methods

In [15]:
function update_auxiliary_state!(
    dg::DGModel,
    m::HeatModel,
    Q::MPIStateArray,
    t::Real,
    elems::UnitRange,
)
    nodal_update_auxiliary_state!(heat_eq_nodal_update_aux!, dg, m, Q, t, elems)
    return true # TODO: remove return true
end;

Compute/update all auxiliary variables at each node. Note that
- `aux.T` is available here because we've specified `T` in `vars_state_auxiliary`

In [16]:
function heat_eq_nodal_update_aux!(
    m::HeatModel,
    state::Vars,
    aux::Vars,
    t::Real,
)
    aux.T = state.ρcT / m.ρc
end;

Since we have second-order fluxes, we must tell `ClimateMachine` to compute the gradient of `ρcT`. Here, we specify how `ρcT` is computed. Note that
 - `transform.ρcT` is available here because we've specified `ρcT` in `vars_state_gradient`

In [17]:
function compute_gradient_argument!(
    m::HeatModel,
    transform::Vars,
    state::Vars,
    aux::Vars,
    t::Real,
)
    transform.ρcT = state.ρcT
end;

Specify where in `diffusive::Vars` to store the computed gradient from `compute_gradient_argument!`. Note that:
 - `diffusive.α∇ρcT` is available here because we've specified `α∇ρcT` in `vars_state_gradient_flux`
 - `∇transform.ρcT` is available here because we've specified `ρcT`  in `vars_state_gradient`

In [18]:
function compute_gradient_flux!(
    m::HeatModel,
    diffusive::Vars,
    ∇transform::Grad,
    state::Vars,
    aux::Vars,
    t::Real,
)
    diffusive.α∇ρcT = m.α * ∇transform.ρcT
end;

We do no have sources, nor non-diffusive fluxes.

In [19]:
function source!(m::HeatModel, _...) end;
function flux_first_order!(
    m::HeatModel,
    flux::Grad,
    state::Vars,
    aux::Vars,
    t::Real,
) end;

Compute diffusive flux (``F(α, ρcT, t) = -α ∇ρcT`` in the original PDE). Note that:
- `diffusive.α∇ρcT` is available here because we've specified `α∇ρcT` in `vars_state_gradient_flux`

In [20]:
function flux_second_order!(
    m::HeatModel,
    flux::Grad,
    state::Vars,
    diffusive::Vars,
    hyperdiffusive::Vars,
    aux::Vars,
    t::Real,
)
    flux.ρcT -= diffusive.α∇ρcT
end;

### Boundary conditions

Second-order terms in our equations, ``∇⋅(G)`` where ``G = α∇ρcT``, are internally reformulated to first-order unknowns.
Boundary conditions must be specified for all unknowns, both first-order and second-order unknowns which have been reformulated.

The boundary conditions for `ρcT` (first order unknown)

In [21]:
function boundary_state!(
    nf,
    m::HeatModel,
    state⁺::Vars,
    aux⁺::Vars,
    n⁻,
    state⁻::Vars,
    aux⁻::Vars,
    bctype,
    t,
    _...,
)
    if bctype == 1 # bottom
        state⁺.ρcT = m.ρc * m.T_bottom
    elseif bctype == 2 # top
        nothing
    end
end;

The boundary conditions for `ρcT` are specified here for second-order unknowns

In [22]:
function boundary_state!(
    nf,
    m::HeatModel,
    state⁺::Vars,
    diff⁺::Vars,
    aux⁺::Vars,
    n⁻,
    state⁻::Vars,
    diff⁻::Vars,
    aux⁻::Vars,
    bctype,
    t,
    _...,
)
    if bctype == 1 # bottom
        state⁺.ρcT = m.ρc * m.T_bottom
    elseif bctype == 2 # top
        diff⁺.α∇ρcT = -n⁻ * m.flux_top
    end
end;

# Spatial discretization

Prescribe vector of vertical elements (in meters)

In [23]:
velems = collect(0:10) / 10;

Prescribe polynomial order of basis functions in finite elements

In [24]:
N_poly = 5;

Define a grid

In [25]:
grid = SingleStackGrid(MPI, velems, N_poly, FT, Array);

Configure the DG model, based on the PDEs, grid and penalty terms

In [26]:
dg = DGModel(
    m,
    grid,
    CentralNumericalFluxFirstOrder(),
    CentralNumericalFluxSecondOrder(),
    CentralNumericalFluxGradient(),
);

# Time discretization

We'll define the time-step based on the [Fourier number](https://en.wikipedia.org/wiki/Fourier_number)

In [27]:
Δ = min_node_distance(grid)

given_Fourier = FT(0.08);
Fourier_bound = given_Fourier * Δ^2 / m.α;
dt = Fourier_bound

0.0011039800162777238

## Initialize the state vector
This initializes the state vector and allocates memory for the solution in space (`dg` has the model `m`, which describes the PDEs as well as the function used for initialization):

In [28]:
Q = init_ode_state(dg, FT(0));

## Initialize the ODE solver
Here, we use an explicit Low-Storage [Runge-Kutta](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) method

In [29]:
lsrk = LSRK54CarpenterKennedy(dg, Q; dt = dt, t0 = 0);

## Inspect the initial conditions

Let's export a plot of the initial state

In [30]:
output_dir = @__DIR__;

mkpath(output_dir);

z_scale = 100 # convert from meters to cm
z_key = "z"
z_label = "z [cm]"
z = get_z(grid, z_scale)
state_vars = get_vars_from_stack(grid, Q, m, vars_state_conservative);
aux_vars =
    get_vars_from_stack(grid, dg.state_auxiliary, m, vars_state_auxiliary);
all_vars = OrderedDict(state_vars..., aux_vars...);
export_plot_snapshot(
    z,
    all_vars,
    ("ρcT",),
    joinpath(output_dir, "initial_condition.png"),
    z_label,
);

![](initial_condition.png)

It matches what we have in `init_state_conservative!(m::HeatModel, ...)`, so let's continue.

# Solver hooks / callbacks

Define simulation time (all units are SI) and number of outputs over this interval

In [31]:
const timeend = 40;
const n_outputs = 5;

This equates to exports every ceil(Int, timeend/n_outputs) time-step:

In [32]:
const every_x_simulation_time = ceil(Int, timeend / n_outputs);

Create a dictionary for `z` coordinate (and convert to cm) NCDatasets IO:

In [33]:
dims = OrderedDict(z_key => collect(z));

Create a DataFile, which is callable to get the name of each file given a step

In [34]:
output_data = DataFile(joinpath(output_dir, "output_data"));

ClimateMachine's time-steppers provide hooks, or callbacks, which allow users to inject code to be executed at specified intervals. In this callback, the state and aux variables are collected, combined into a single `OrderedDict` and written to a NetCDF file (for each output step `step`).

In [35]:
step = [0];
callback = GenericCallbacks.EveryXSimulationTime(
    every_x_simulation_time,
    lsrk,
) do (init = false)
    state_vars = get_vars_from_stack(grid, Q, m, vars_state_conservative)
    aux_vars = get_vars_from_stack(
        grid,
        dg.state_auxiliary,
        m,
        vars_state_auxiliary;
        exclude = [z_key],
    )
    all_vars = OrderedDict(state_vars..., aux_vars...)
    all_vars = prep_for_io(z_label, all_vars)
    write_data(
        NetCDFWriter(),
        output_data(step[1]),
        dims,
        all_vars,
        gettime(lsrk),
    )
    step[1] += 1
    nothing
end;

# Solve

This is the main "solve" method call. While users do not have access to the time-stepping loop, code and function calls may be injected via `callbacks` keyword argument, which is a `Tuple` of `GenericCallbacks`.

In [36]:
solve!(Q, lsrk; timeend = timeend, callbacks = (callback,));

# Post-processing

Our solution has now been calculated and exported to NetCDF files in `output_dir`. Let's collect them all into a nested dictionary whose keys are the output interval. The next level keys are the variable names, and the values are the values along the grid:

In [37]:
all_data = collect_data(output_data, step[1]);

To get `T` at ``t=0``, we can use `T_at_t_0 = all_data[0]["T"][:]`

In [38]:
@show keys(all_data[0])

keys(all_data[0]) = ["T", "t", "simtime", "z", "ρcT"]


Base.KeySet for a Dict{String,Array{Any,1}} with 5 entries. Keys:
  "T"
  "t"
  "simtime"
  "z"
  "ρcT"

Let's plot the solution:

In [39]:
export_plot(z, all_data, ("ρcT",), joinpath(output_dir, "solution_vs_time.png"), z_label);

![](solution_vs_time.png)

The results look as we would expect: a fixed temperature at the bottom is resulting in heat flux that propagates up the domain. To run this file, and inspect the solution in `all_data`, include this tutorial in the Julia REPL with:

```julia
include(joinpath("tutorials", "Land", "Heat", "heat_equation.jl"))
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*